<a href="https://colab.research.google.com/github/ilyayath/imdb-classification/blob/main/2_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Що таке GloVe

* Це **заздалегідь навчені векторні представлення слів** (embedding).
* Кожне слово має свій вектор з **50 чисел** (у твоєму файлі).
* Вектори схожих слів розташовані близько одне до одного у багатовимірному просторі.

Приклад (спрощено):

```
king → [0.1, 0.3, -0.2, ...]  
queen → [0.09, 0.28, -0.19, ...]  
```

---

### 2. Як це використати для твого завдання

У тебе вхід — **речення або відгук**, а GloVe працює на рівні **слів**.
Тому потрібно:

1. Розбити текст на слова.
2. Для кожного слова знайти його вектор у GloVe.
3. Усереднити всі вектори, щоб отримати **один вектор для всього тексту**.
4. Використати цей вектор як вхід для моделі ML (LogReg, SVM, XGBoost і т.д.).

---

### 3. Приклад логіки на словах

Текст:

```
I love this movie
```

GloVe дає:

```
"I"    → [0.12,  0.03, -0.20, ...]  
"love" → [0.55, -0.12,  0.33, ...]  
"this" → [-0.01, 0.22,  0.05, ...]  
"movie"→ [0.20,  0.18, -0.08, ...]  
```

Ти береш **середнє значення кожної координати**:

```
[ (0.12+0.55-0.01+0.20)/4 , ... ]
```

І отримаєш **вектор для всього речення**.

---

### 4. Потім

* Кожен відгук → перетворюєш у такий вектор.
* Отримаєш матрицю `n_samples × 50` (бо 50 — розмір embedding).
* Цю матрицю подаєш у будь-який ML алгоритм.




In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report

In [ ]:
train = pd.read_csv('/content/train_df.csv')
test = pd.read_csv('/content/test_df.csv')

In [ ]:
embeddings_index = {}

In [ ]:
with open("/content/drive/MyDrive/Copy of wiki", encoding="utf-8") as file:
  for line in file:
    line = line.strip()
    if not line:
      continue
    values = line.split()
    word = values[0]
    try:
      vector = np.asarray([float(x) for x in values[1:]],dtype='float32')
      embeddings_index[word] = vector
    except ValueError:
      continue

In [ ]:
train

,text,label,len_of_text
0,rent video store controversy surround first re...,0,288
1,curious yellow risible pretentious steam pile ...,0,214
2,avoid make type film future film interest expe...,0,93
3,film probably inspire godard masculin féminin ...,0,118
4,oh brother hear ridiculous film umpteen year t...,0,311
...,...,...,...
24995,hit time well categorise australian cult film ...,1,108
24996,love movie like another time try explain virtu...,1,183
24997,film sequel barry mckenzie hold two great come...,1,134
24998,adventure barry mckenzie started life satirica...,1,717


In [ ]:
X_train_vectors = np.array([
    np.mean([embeddings_index[token] for token in text.split() if token in embeddings_index],axis=0)
    for text in train['text']
    ])
X_test_vectors = np.array([
    np.mean([embeddings_index[token] for token in text.split() if token in embeddings_index],axis=0)
    for text in test['text']
    ])

In [ ]:
pipelines = {
    "Logistic Regression": Pipeline([
        ('clf', LogisticRegression(max_iter=200))
    ]),
    "Linear SVC": Pipeline([
        ('clf', LinearSVC())
    ])
}

In [ ]:
result = []

In [ ]:
for model, pipeline in pipelines.items():
  pipeline.fit(X_train_vectors,train['label'])
  preds = pipeline.predict(X_test_vectors)
  accuracy = accuracy_score(test['label'],preds)
  precision = precision_score(test['label'],preds)
  recall = recall_score(test['label'],preds)
  f1 = f1_score(test['label'],preds)
  result.append({'Model': model,
                 'Accuracy' : accuracy,
                 'Precision' : precision,
                 'Recall' : recall,
                 'F1' : f1
                 })



In [ ]:
pd.DataFrame(result)

,Model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.75608,0.759022,0.7504,0.754687
1,Linear SVC,0.75724,0.760596,0.7508,0.755666
